CSE 5243 - March 31, 2018

# Lab 4 — Association Rule Mining

In this lab, you'll preprocess a dataset and use the [MLxtend](http://rasbt.github.io/mlxtend/) machine
learning library to mine frequent patterns and association rules.

Please follow all instructions carefully. There are 16 steps. When you are finished, you
will submit your work via Carmen as usual.

**This lab is due on Tuesday, April 3, 2018, at 11:59 PM ET.**


### BEFORE YOU BEGIN: Install the MLxtend machine learning library.

If you're using anaconda, this is realy easy! At the command prompt, enter the following:

```
$ conda install -c conda-forge mlxtend
```

Answer 'yes' to install any dependencies or updates. That's it! If you run into trouble,
[email me right away](mailto:burkhardt.5@osu.edu)!

In [1]:
# These are the libraries you are most likely to need, but you may include others as well.
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

## Part 1. Download the data.

For this lab, you will use the [Zoo dataset](https://archive.ics.uci.edu/ml/datasets/Zoo), which 
is available for download at the UCI Machine Learning Repository.

### 1. Download the file (`zoo.data`). The file should contain 101 rows $\times$ 18 columns.


In [2]:
# If you don't already have a copy of the Iris data file, you can download it here.
DATAURL = "https://archive.ics.uci.edu/ml/machine-learning-databases/zoo/zoo.data"
import urllib
urllib.request.urlretrieve (DATAURL, "zoo.data")

('zoo.data', <http.client.HTTPMessage at 0x97fac18>)

## Part 2. Preprocessing

### 2. Use the input data to generate a new data frame with 24 columns, as follows:

| Column No. | Column Name | Description |
|-|-|-|
|1|name| The name of the animal (string)|
|2-16|hair, feathers, ...| All (15) asymmetric binary attributes from the original data file (hair, feathers, etc.)|
|17|has_legs| Does the animal have legs? (1=yes, 0=no) |
|18|mammal|Is the animal a mammal? (1=yes, 0=no) |
|19|bird|Is the animal a bird? (1=yes, 0=no) |
|20|reptile|Is the animal a reptile? (1=yes, 0=no) |
|21|fish|Is the animal a fish? (1=yes, 0=no) |
|22|amphibian|Is the animal an amphibian? (1=yes, 0=no) |
|23|insect|Is the animal an insect? (1=yes, 0=no) |
|24|mollusk|Is the animal a mollusk? (1=yes, 0=no) |

In [3]:
labels=["animaname","hair","feathers","eggs","milk","airborne","aquatic","predator","toothed","backbone","breathes","venomous","fins","legs","tail","domestic","catsize","type"]
df=pd.read_csv("zoo.data", sep=',',engine='python', names=labels)

# Change legs to has_leg
legs = df["legs"]
df = df.drop("legs",1)
for i in range(legs.size):
    if legs[i] > 0:
        legs[i] =1
df["has_legs"]=legs

#Change type to seperate types
tp = df["type"]
types = np.zeros((df["type"].size,7),dtype=int)
df = df.drop("type",1)
for i in range(tp.size):
    types[i][tp[i]-1]=1
typeLables = ["mammal","bird","reptile","fish","amphibian","insect","mollusk"]
tmpdf = pd.DataFrame.from_records(types,columns=typeLables)
df=df.join(tmpdf)
df.sample(5)

,animaname,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,...,domestic,catsize,has_legs,mammal,bird,reptile,fish,amphibian,insect,mollusk
5,buffalo,1,0,0,1,0,0,0,1,1,...,0,1,1,1,0,0,0,0,0,0
21,duck,0,1,1,0,1,1,0,0,1,...,0,0,1,0,1,0,0,0,0,0
20,dove,0,1,1,0,1,0,0,0,1,...,1,0,1,0,1,0,0,0,0,0
8,catfish,0,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
43,lark,0,1,1,0,1,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0


### 3. Save the data frame to a CSV file and give it a descriptive name (e.g. `zoo_basket.csv`).

In [4]:
df.to_csv("zoo_basket.csv")

## Part 3. Generate Frequent Patterns

### 4. Use the [MLxtend **apriori**](http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/#apriori) function to find frequent patterns with $min\_support=0.5$ and $max\_len=5$.

In [5]:
apriori(df.drop("animaname",1),min_support=0.5,max_len=5.0)

,support,itemsets
0,0.584158,[2]
1,0.554455,[6]
2,0.603960,[7]
3,0.821782,[8]
4,0.792079,[9]
5,0.742574,[12]
6,0.772277,[15]
7,0.603960,"[7, 8]"
8,0.514851,"[7, 12]"
9,0.683168,"[8, 9]"


### 5. How many frequent itemsets were generated? List all the frequent 3-itemsets.

21 frequent itemsets were generated.

Frequent 3-itemsets:

* 7,8,12: toothed, backbone, tail
* 8,9,12: backbone, breathes, tail
* 8,9,15: backbone, breathes, has_legs
* 8,12,15: backbone, tail, has_legs
* 9,12,15: breathes, tail, has_legs


### 6. Experiment with different values of $min\_support$ and $max\_len$.
* Are you able to find non-zero values for the parameters $min\_support$ and $max\_len$
  that yield frequent itemsets containing all 7 of the animal type items?
* How many frequent patterns were generated using these parameters?

Ans:
* It is not possible to yield one frequent itemset containing all 7 of the animal types with *min_support* > 0. Because for any subject, there could only be one animal type.
* It is possible to generate frequent 1-itemsets like the example below. There are 23 patterns in all.

In [6]:
testdf=df.drop("animaname",1)
apriori(testdf,min_support=0.039, max_len=1)

,support,itemsets
0,0.425743,[0]
1,0.198020,[1]
2,0.584158,[2]
3,0.405941,[3]
4,0.237624,[4]
5,0.356436,[5]
6,0.554455,[6]
7,0.603960,[7]
8,0.821782,[8]
9,0.792079,[9]


## Part 4. Association Rule Mining

### 7. What is the maximum number of association rules that can be extracted from this dataset, including rules that have zero support?

In [7]:
fact = np.math.factorial
numItemset = 0;
for i in range(23):
    numItemset = numItemset + fact(23)/(fact(i)*fact(23-i))
numItemset
numAs = numItemset*(numItemset-1)
numAs

70368719011842.0

There are totally 8388607 itemsets, so the maximum number of association rules are $8388607*8388606=70368719011842$ (excluding rule pointing to the itemset itself).

### 8. Regenerate frequent patterns using $min\_support=0.1$ and $max\_len=5$.

In [8]:
apr = apriori(testdf,min_support=0.1, max_len=5)

### 9. Use the [MLxtend **association_rules**](http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/#association_rules) function to find association rules with $min\_threshold=0.7$.

In [9]:
rules = association_rules(apr, min_threshold=0.7)
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(0),(3),0.425743,0.405941,0.386139,0.906977,2.234260,0.213312,6.386139
1,(3),(0),0.405941,0.425743,0.386139,0.951220,2.234260,0.213312,11.772277
2,(0),(7),0.425743,0.603960,0.376238,0.883721,1.463210,0.119106,3.405941
3,(0),(8),0.425743,0.821782,0.386139,0.906977,1.103670,0.036271,1.915842
4,(0),(9),0.425743,0.792079,0.425743,1.000000,1.262500,0.088521,inf
5,(0),(12),0.425743,0.742574,0.326733,0.767442,1.033488,0.010587,1.106931
6,(0),(15),0.425743,0.772277,0.415842,0.976744,1.264758,0.087050,9.792079
7,(0),(16),0.425743,0.405941,0.386139,0.906977,2.234260,0.213312,6.386139
8,(16),(0),0.405941,0.425743,0.386139,0.951220,2.234260,0.213312,11.772277
9,(1),(2),0.198020,0.584158,0.198020,1.000000,1.711864,0.082345,inf


### 10. How many association rules were found?

There are 8584 rules in all.

### 11. Construct a contingency table for the rule

$$ \{predator, catsize, toothed, mammal\} \rightarrow \{tail\} $$

In [10]:
contingency = [[0,0],[0,0]]
for i in range(0,df.shape[0]):
    x = (df["predator"][i]==1 and df["catsize"][i]==1 and df["toothed"][i]==1 and df["mammal"][i]==1)
    y = (df["tail"][i] == 1)
    if x and y:
        contingency[0][0] += 1
    elif x and not y:
        contingency[0][1] += 1
    elif not x and y:
        contingency[1][0] += 1
    else:
        contingency[1][1] += 1
contingency

[[15, 4], [60, 22]]

Thus the contingency table for this rule is

| | $Y$ | $\neg Y$   |
|------|------|------|
|   $X$  | 15| 4 |
|   $\neg X$  | 60| 22|

### 12. Use the contingency table to compute support for the rule. Does your result agree with the output from the *association_rules* function?
$S(X \rightarrow Y) = \frac{\sigma(X \cup Y)}{N} = \frac{15}{15+4+60+22} = 14.85\%$

In [11]:
for i in range(0,8584):
    if rules['antecedants'][i] == frozenset({6,7,14,16}) and rules['consequents'][i] == frozenset({12}):
        print(rules['support'][i])

0.148514851485


Yes, the result calculated from the contingency table agree with the output from association_rules function.

### 13. List the "most interesting" rules—those with the highest measures of confidence, lift, and leverage. If there are ties (i.e. more than one rule sharing the highest value) then show them all.

In [12]:
interestingRules = pd.DataFrame()

maxConfi = max(rules['confidence'])
maxLift = max(rules['lift'])
maxLeverage = max(rules['leverage'])

for i in range(0,8584):
    if rules['confidence'][i] == maxConfi or rules['lift'][i] == maxLift or rules['leverage'][i] == maxLeverage:
        interestingRules = interestingRules.append(rules.loc[i])

interestingRules

,antecedants,antecedent support,confidence,consequent support,consequents,conviction,leverage,lift,support
4,(0),0.425743,1.0,0.792079,(9),inf,0.088521,1.262500,0.425743
9,(1),0.198020,1.0,0.584158,(2),inf,0.082345,1.711864,0.198020
11,(1),0.198020,1.0,0.821782,(8),inf,0.035291,1.216867,0.198020
12,(1),0.198020,1.0,0.792079,(9),inf,0.041172,1.262500,0.198020
13,(1),0.198020,1.0,0.742574,(12),inf,0.050975,1.346667,0.198020
14,(1),0.198020,1.0,0.772277,(15),inf,0.045094,1.294872,0.198020
15,(1),0.198020,1.0,0.198020,(17),inf,0.158808,5.050000,0.198020
16,(17),0.198020,1.0,0.198020,(1),inf,0.158808,5.050000,0.198020
21,(17),0.198020,1.0,0.584158,(2),inf,0.082345,1.711864,0.198020
22,(19),0.128713,1.0,0.584158,(2),inf,0.053524,1.711864,0.128713


## Part 5. Association Rules as Predictors

### 14. How many association rules were found in which the consequent contains exactly one element, and that element is one of the 7 animal types?

(Let's call this "Ruleset 2". Save these to another data frame or matrix, and answer
the remaining questions in this section based ONLY ON RULESET2.)


In [13]:
ruleSet2 = pd.DataFrame()

for i in range(0,8584):
    if len(rules['consequents'][i]) == 1 and next(iter(rules['consequents'][i]))>=16:
        ruleSet2 = ruleSet2.append(rules.loc[i])

ruleSet2

,antecedants,antecedent support,confidence,consequent support,consequents,conviction,leverage,lift,support
7,(0),0.425743,0.906977,0.405941,(16),6.386139,0.213312,2.234260,0.386139
15,(1),0.198020,1.000000,0.198020,(17),inf,0.158808,5.050000,0.198020
31,(3),0.405941,1.000000,0.405941,(16),inf,0.241153,2.463415,0.405941
75,(11),0.168317,0.764706,0.128713,(19),3.702970,0.107048,5.941176,0.128713
85,(14),0.435644,0.727273,0.405941,(16),2.178218,0.139986,1.791574,0.316832
120,"(0, 3)",0.386139,1.000000,0.405941,(16),inf,0.229389,2.463415,0.386139
131,"(0, 6)",0.198020,1.000000,0.405941,(16),inf,0.117636,2.463415,0.198020
151,"(0, 7)",0.376238,1.000000,0.405941,(16),inf,0.223507,2.463415,0.376238
166,"(0, 8)",0.386139,1.000000,0.405941,(16),inf,0.229389,2.463415,0.386139
179,"(0, 9)",0.425743,0.906977,0.405941,(16),6.386139,0.213312,2.234260,0.386139


There are 317 rules that satisfy the conditions.

### 15. How many rules are there for each animal type?

In [14]:
animalTypes = {}
for i in ruleSet2['consequents']:
    animalType = next(iter(i))
    if animalType not in animalTypes:
        animalTypes[animalType] = 1
    else:
        animalTypes[animalType] += 1
animalTypes

{16: 215, 17: 71, 19: 31}

There are 215 rules for mammals, 71 rules for birds, and 31 rules for fish.

## 6. Thought Experiment

Don't implement anything for this section. Just write out your answer in a markdown cell.

### 16. Explain how you might use association rules to suggest an animal's type based on the other attributes. Based on your observations, what changes would you make in the frequent itemset generation and rule generation steps?

First we can find all the rules that only has one consequent, and this consequent is a animal's type, as the "Ruleset 2" in section 5. Then from this rule set we can select a set of association rules that can achieve a good classification accuracy and don't conflict with each other first. Then for every animal with known attribute values, we can find a rule that the antecedents match this animal's features, and the consequent of this rule will be the predicted animal's type.

For the frequent itemset generation part, we can exclude the animal type attributes befor the generation, and then
during the rule generation, we only consider antecedants from the frequent itemsets and use the animal types seperately as the consequences. 

## 7. Submit your work

**Submit your completed notebook, along with the data file you created in Part 2, via Carmen as LAB 4.**